In [1]:
import pandas as pd
import database_connect as db_connect
import stock_marker_binance
import numpy as np
from datetime import datetime, timedelta

In [2]:
CONFIG = db_connect.load_config()

In [3]:
def hours_to_milliseconds(hours):
    return hours * 60 * 60 * 1000

In [4]:
def insert_data_to_db(conn, df):
    with conn.cursor() as cursor:
        for row in df.itertuples():
            query = f"INSERT INTO trading_bot_historydataeth (timestamp, open, high, low, close, volume, \
            close_time, quote_asset_volume, num_trades) \
            VALUES ('{row.timestamp}', {row.open}, {row.high}, {row.low}, {row.close}, {row.volume}, \
            {row.close_time}, {row.quote_asset_volume}, {row.num_trades})"
            cursor.execute(query)
            conn.commit()

### Adding historical data to the database.

In [ ]:
df = pd.read_csv('ETH_1h_history as of 09-01-2024.csv', index_col=0)

In [25]:
df.head()

,timestamp,open,high,low,close,volume,close_time,quote_asset_volume,num_trades
0,01/01/2020 00:00,129.16,129.19,128.68,128.87,7769.17336,1.577840e+12,1.000930e+06,2504
1,01/01/2020 01:00,128.87,130.65,128.78,130.64,11344.65516,1.577840e+12,1.474278e+06,4885
2,01/01/2020 02:00,130.63,130.98,130.35,130.85,7603.35623,1.577850e+12,9.940256e+05,3046
3,01/01/2020 03:00,130.85,130.89,129.94,130.20,4968.55433,1.577850e+12,6.473610e+05,2818
4,01/01/2020 04:00,130.21,130.74,130.15,130.20,3397.90747,1.577850e+12,4.430067e+05,2264


In [27]:
conn = db_connect.connect(CONFIG)

Connected to the PostgreSQL server.


In [28]:
for row in df.itertuples():
    print(tuple(row))
    break

(0, '01/01/2020 00:00', 129.16, 129.19, 128.68, 128.87, 7769.17336, 1577840000000.0, 1000929.742, 2504)


In [29]:
# insert_data_to_db(conn, df) # Commented to avoid inserting data to db

In [32]:
conn.close()

### Update database with fresh data from Binance

In [5]:
# Every hour the script updates the database with the latest data from the API.
# The script is run using a cron job.

# Identifying the last timestamp in the database and using that as the starting point for the API call.

In [6]:
# Identifying the last timestamp in the database and using that as the starting point for the API call.
conn = db_connect.connect(CONFIG)

Connected to the PostgreSQL server.


In [7]:
with conn.cursor() as cursor:
    cursor.execute("SELECT MAX(timestamp) FROM trading_bot_historydataeth")
    result = cursor.fetchone()
    print(result)
conn.close()

(datetime.datetime(2024, 3, 2, 18, 0, tzinfo=datetime.timezone(datetime.timedelta(seconds=18000))),)


In [8]:
date_from =  result[0]- timedelta(hours=385)
date_from = datetime.strftime(date_from, '%Y-%m-%d %H:%M')

In [9]:
INTERVAL_START = result[0]

print(INTERVAL_START)

INTERVAL_START = INTERVAL_START.replace(tzinfo=None)
print(INTERVAL_START -  stock_marker_binance.UNIX_TIME_START)
INTERVAL_START = int((INTERVAL_START -  stock_marker_binance.UNIX_TIME_START).total_seconds()) * 1000 + 1
print(INTERVAL_START)




2024-03-02 18:00:00+05:00
19784 days, 18:00:00
1709402400001


In [11]:
url = 'https://api3.binance.com/api/v3/time'

df_time_stop = pd.read_json(url, typ='series')

TIME_STOP = df_time_stop.iloc[0]
print(TIME_STOP)

# TIME_STOP = int ( time.mktime(TIME_STOP.timetuple())*1000)
TIME_STOP = int(TIME_STOP.timestamp()) * 1000
print(TIME_STOP)

2024-07-09 07:58:02.366000
1720511882000


In [12]:
SYMBOL = 'ETHUSDT'
INTERVAL = '1h'
df_total = pd.DataFrame()

In [13]:
while INTERVAL_START < TIME_STOP:
      df=stock_marker_binance.import_stock_data(SYMBOL, INTERVAL, startTime =INTERVAL_START,  limit=984)
      df.rename(
          columns={0:'timestamp', 1:'open', 2:'high', 3:'low', 4:'close',5:'volume', 6:'close_time', 7:'quote_asset_volume',
                8:'num_trades', 9:'taker_buy_base_asset_volume',10:'taker_buy_quote_asset_volume', 11:'unused_field'},
         inplace=True)
      df.drop(columns=['taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'unused_field'], inplace=True)
      if len(df_total)>0:
            df_total = pd.concat([df_total, df], ignore_index=True)
      else:
            df_total = df.copy()
      INTERVAL_START = INTERVAL_START + hours_to_milliseconds(984)
      print(len(df_total))

984
1968
2952
3085


In [14]:
df_total.head(10)

,timestamp,open,high,low,close,volume,close_time,quote_asset_volume,num_trades
0,2024-03-02 19:00:00,3430.51,3431.03,3423.79,3426.19,7455.7281,1709409599999,2.555149e+07,25380
1,2024-03-02 20:00:00,3426.20,3431.31,3404.40,3406.29,13255.7449,1709413199999,4.522597e+07,37266
2,2024-03-02 21:00:00,3406.30,3413.40,3402.56,3408.19,6862.3405,1709416799999,2.338225e+07,27069
3,2024-03-02 22:00:00,3408.19,3409.88,3402.99,3404.00,4882.2389,1709420399999,1.663166e+07,20574
4,2024-03-02 23:00:00,3404.01,3421.48,3404.00,3421.40,6500.4525,1709423999999,2.220275e+07,24546
5,2024-03-03 00:00:00,3421.39,3431.00,3409.10,3427.86,13926.4053,1709427599999,4.764907e+07,47448
6,2024-03-03 01:00:00,3427.86,3434.00,3422.42,3430.76,10359.4624,1709431199999,3.551396e+07,34829
7,2024-03-03 02:00:00,3430.75,3430.97,3410.81,3416.10,8935.3151,1709434799999,3.054830e+07,32807
8,2024-03-03 03:00:00,3416.11,3431.86,3415.16,3431.56,8976.9543,1709438399999,3.075216e+07,28175
9,2024-03-03 04:00:00,3431.57,3446.23,3428.78,3440.60,13711.6275,1709441999999,4.715641e+07,34325


In [15]:
conn = db_connect.connect(CONFIG)
insert_data_to_db(conn, df_total)
conn.close()

Connected to the PostgreSQL server.


### Recalculate the additional columns

In [16]:
conn = db_connect.connect(CONFIG)

Connected to the PostgreSQL server.


In [17]:
with conn.cursor() as cursor:
    cursor.execute(f"SELECT id, timestamp, open,	high,	low,	close,	volume,	close_time, \
                    quote_asset_volume,	num_trades \
                    FROM trading_bot_historydataeth \
                    WHERE timestamp > '{date_from}' \
                    ORDER BY timestamp DESC")
    result = cursor.fetchall()
conn.close()

In [18]:
df = pd.DataFrame(result, columns=['id', 'timestamp', 'open', 'high', 'low', 
                                   'close', 'volume', 'close_time', 'quote_asset_volume', 'num_trades'])

In [19]:
df_result = stock_marker_binance.sorting_timestamp(df)

df_result['hour_sin'] = np.sin(df_result['timestamp'].dt.hour / 24 * 2 * np.pi)
df_result['hour_cos'] = np.cos(df_result['timestamp'].dt.hour / 24 * 2 * np.pi)

df_result['day_sin'] = np.sin(df_result['timestamp'].dt.day / 30 * 2 * np.pi)
df_result['day_cos'] = np.cos(df_result['timestamp'].dt.day / 30 * 2 * np.pi)

df_result['mon_sin'] = np.sin(df_result['timestamp'].dt.month / 12 * 2 * np.pi)
df_result['mon_cos'] = np.cos(df_result['timestamp'].dt.month / 12 * 2 * np.pi)

df_result['weekday_sin'] = np.sin(df_result['timestamp'].dt.weekday / 7 * 2 * np.pi)
df_result['weekday_cos'] = np.cos(df_result['timestamp'].dt.weekday / 7 * 2 * np.pi)

df_result['year'] = df_result['timestamp'].dt.year

In [20]:
df_result = stock_marker_binance.std(df_result, ['close', 'quote_asset_volume', 'num_trades'])
print("Complete adding STDs")

df_result = stock_marker_binance.adding_MAs(df_result, ['close', 'quote_asset_volume', 'num_trades'])
print("Complete adding MAs")


df_result = stock_marker_binance.adding_ratio(df_result, ['close', 'quote_asset_volume', 'num_trades'], ['MA'])
df_result = stock_marker_binance.adding_ratio(df_result, ['close'], ['STD'])
print("Complete adding ratios")

df_result = stock_marker_binance.rsi(df_result, ['close', 
                                                'quote_asset_volume', 'num_trades'
                                                 ])
print("Complete adding RSI")


df_result = df_result.drop(columns=['timestamp','open', 'high', 'low' ,'close_time', 'volume',
                                    'close',  'quote_asset_volume', 'num_trades'
                                    ])
df_result=df_result.dropna()
df_result.reset_index(inplace=True, drop=True)

Complete adding STDs
Complete adding MAs
Complete adding ratios
Complete adding RSI


c:\Users\user\Documents\GitHub\BinanceTradingBot\stock_marker_binance.py:278: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['RSI_'+ str(period) + '_' + col_label] = 100.0 - (100.0 / (1.0 + RS))
c:\Users\user\Documents\GitHub\BinanceTradingBot\stock_marker_binance.py:278: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['RSI_'+ str(period) + '_' + col_label] = 100.0 - (100.0 / (1.0 + RS))
c:\Users\user\Documents\GitHub\BinanceTradingBot\stock_marker_binance.py:278: PerformanceWarning: DataFrame is highly fragmented.  This is u

In [21]:
df_result

,id,hour_sin,hour_cos,day_sin,day_cos,mon_sin,mon_cos,weekday_sin,weekday_cos,year,...,RSI_192_quote_asset_volume,RSI_384_quote_asset_volume,RSI_3_num_trades,RSI_6_num_trades,RSI_12_num_trades,RSI_24_num_trades,RSI_48_num_trades,RSI_96_num_trades,RSI_192_num_trades,RSI_384_num_trades
0,40164,-1.000000,-1.836970e-16,0.406737,0.913545,1.0,6.123234e-17,-0.974928,-0.222521,2024,...,49.825310,49.893149,11.158123,35.679243,36.017486,43.535167,44.672629,49.219163,49.733104,49.731794
1,40165,-0.965926,2.588190e-01,0.406737,0.913545,1.0,6.123234e-17,-0.974928,-0.222521,2024,...,49.976689,49.943128,27.121784,44.399138,45.018855,47.742739,47.466715,49.236079,49.930799,49.823988
2,40166,-0.866025,5.000000e-01,0.406737,0.913545,1.0,6.123234e-17,-0.974928,-0.222521,2024,...,50.136208,50.035073,55.972981,36.433623,45.244258,48.304377,49.007289,49.457877,50.145411,49.901474
3,40167,-0.707107,7.071068e-01,0.406737,0.913545,1.0,6.123234e-17,-0.974928,-0.222521,2024,...,50.024379,49.830994,69.695078,39.858948,38.905577,46.833612,43.366752,49.553520,50.077308,49.700241
4,40168,-0.500000,8.660254e-01,0.406737,0.913545,1.0,6.123234e-17,-0.974928,-0.222521,2024,...,49.618133,49.907695,41.591434,32.928011,45.127216,48.480563,46.020630,49.606711,49.735640,49.836921
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3081,43245,0.707107,7.071068e-01,0.951057,-0.309017,-0.5,-8.660254e-01,0.781831,0.623490,2024,...,50.055015,50.117304,51.032229,47.296866,30.042424,48.723224,50.503626,42.860158,50.230075,50.246413
3082,43246,0.866025,5.000000e-01,0.951057,-0.309017,-0.5,-8.660254e-01,0.781831,0.623490,2024,...,50.169410,50.166968,12.813266,56.329877,42.800280,48.958804,50.739195,45.365593,50.311006,50.314735
3083,43247,0.965926,2.588190e-01,0.951057,-0.309017,-0.5,-8.660254e-01,0.781831,0.623490,2024,...,49.839658,50.045054,25.687161,57.905786,44.624308,49.395598,50.638276,47.499418,50.227886,50.263778
3084,43248,1.000000,6.123234e-17,0.951057,-0.309017,-0.5,-8.660254e-01,0.781831,0.623490,2024,...,50.094535,50.148230,72.790024,56.017756,53.172843,49.782108,51.496140,48.279932,50.460092,50.360928


In [22]:
drop_cols_list =[]
for col in df_result.columns:
    if col[:2]=="MA" or col[:3]=="STD":
        drop_cols_list.append(col)
df_result.drop(columns=drop_cols_list, inplace=True)

In [23]:
df_result

,id,hour_sin,hour_cos,day_sin,day_cos,mon_sin,mon_cos,weekday_sin,weekday_cos,year,...,RSI_192_quote_asset_volume,RSI_384_quote_asset_volume,RSI_3_num_trades,RSI_6_num_trades,RSI_12_num_trades,RSI_24_num_trades,RSI_48_num_trades,RSI_96_num_trades,RSI_192_num_trades,RSI_384_num_trades
0,40164,-1.000000,-1.836970e-16,0.406737,0.913545,1.0,6.123234e-17,-0.974928,-0.222521,2024,...,49.825310,49.893149,11.158123,35.679243,36.017486,43.535167,44.672629,49.219163,49.733104,49.731794
1,40165,-0.965926,2.588190e-01,0.406737,0.913545,1.0,6.123234e-17,-0.974928,-0.222521,2024,...,49.976689,49.943128,27.121784,44.399138,45.018855,47.742739,47.466715,49.236079,49.930799,49.823988
2,40166,-0.866025,5.000000e-01,0.406737,0.913545,1.0,6.123234e-17,-0.974928,-0.222521,2024,...,50.136208,50.035073,55.972981,36.433623,45.244258,48.304377,49.007289,49.457877,50.145411,49.901474
3,40167,-0.707107,7.071068e-01,0.406737,0.913545,1.0,6.123234e-17,-0.974928,-0.222521,2024,...,50.024379,49.830994,69.695078,39.858948,38.905577,46.833612,43.366752,49.553520,50.077308,49.700241
4,40168,-0.500000,8.660254e-01,0.406737,0.913545,1.0,6.123234e-17,-0.974928,-0.222521,2024,...,49.618133,49.907695,41.591434,32.928011,45.127216,48.480563,46.020630,49.606711,49.735640,49.836921
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3081,43245,0.707107,7.071068e-01,0.951057,-0.309017,-0.5,-8.660254e-01,0.781831,0.623490,2024,...,50.055015,50.117304,51.032229,47.296866,30.042424,48.723224,50.503626,42.860158,50.230075,50.246413
3082,43246,0.866025,5.000000e-01,0.951057,-0.309017,-0.5,-8.660254e-01,0.781831,0.623490,2024,...,50.169410,50.166968,12.813266,56.329877,42.800280,48.958804,50.739195,45.365593,50.311006,50.314735
3083,43247,0.965926,2.588190e-01,0.951057,-0.309017,-0.5,-8.660254e-01,0.781831,0.623490,2024,...,49.839658,50.045054,25.687161,57.905786,44.624308,49.395598,50.638276,47.499418,50.227886,50.263778
3084,43248,1.000000,6.123234e-17,0.951057,-0.309017,-0.5,-8.660254e-01,0.781831,0.623490,2024,...,50.094535,50.148230,72.790024,56.017756,53.172843,49.782108,51.496140,48.279932,50.460092,50.360928


In [24]:
cols = list(df_result.columns)

In [25]:
cols.remove('id')

In [26]:
conn = db_connect.connect(CONFIG)

Connected to the PostgreSQL server.


In [27]:
for index, row in df_result.iterrows():
    update_sql = "UPDATE trading_bot_historydataeth SET "
    for col in cols:
        update_sql += f'"{col}" = {row[col]}, '
    update_sql = update_sql[:-2] # Remove the last comma and space
    update_sql += f' WHERE id = {int(row["id"])}  and "hour_sin" IS NULL'
    with conn.cursor() as cursor:
        cursor.execute(update_sql)
        conn.commit()
conn.close()

In [28]:
conn.close()